# L41: Lab 1 - Getting started with kernel tracing - I/O

The goals of this lab are to:
    
- Introduce you to our experimental environment and DTrace.
- Have you explore user-kernel interactions via system calls and traps.
- Gain experience tracing I/O behaviour in UNIX.
- Develop an intuition about DTrace's probe effect.

You will do this using DTrace to analyse the behaviour of a potted, kernel-intensive block-I/O benchmark.

---
## Block-I/O benchmark

### Compiling the benchmark

In [ ]:
# Extract and build the benchmark
!bmake -C io

### Running the benchmark

Once built, you can run the benchmark binaries as follows (with the command-line arguments specifying various benchmark parameters):

In [ ]:
# Execute the io-static benchmark displaying the command line options
!io/io-static

In [ ]:
# Execute the io-dynamic benchmark displaying the command line options
!io/io-dynamic

### Example benchmark commands

The following example creates a default-sized benchmark data file (the benchmark data file is stored in the `/data` directory on the BeagleBone Black SD card):

In [ ]:
# Example benchmark command
print("Creating file to run benchmark")

!io/io-static -c iofile

print("Completed")

The following example runs a simple `read()` benchmark on the benchmark data file, printing additional information (`-v`) about the benchmark run:

In [ ]:
# Example benchmark command
print("Running benchmark")

!io/io-static -v -r iofile

print("Completed")

The following example runs a simple `write()` benchmark on the data file, printing additional information (`-v`) about the benchmark run:

In [ ]:
# Example benchmark command
print("Running benchmark")

!io/io-static -v -w iofile

print("Completed")

The following example runs a simple write() benchmark multiple times printing out the average measured I/O bandwidth (KiBytes/sec):

In [ ]:
# Example benchmark command
import numpy as np
print("Running benchmark")

# Run the benchmark multiple times storing each measure bandwidth in values[]
values = []

for trials in range(0, 10):
    output = !io/io-static -w iofile
    print(trials, output)
    values.append(float(output[1].split()[1]))
    
# Compute and print the average I/O bandwidth
average_bw = np.mean(values)
median_bw = np.median(values)
q1_bw = np.quantile(values, 0.25)
q3_bw = np.quantile(values, 0.75)
print("Average bandwidth = {} KiBytes/sec".format(average_bw))
print("Q1 bandwidth = {} KyBytes/sec".format(q1_bw))
print("Median bandwidth = {} KyBytes/sec".format(median_bw))
print("Q3 = {} KyBytes/sec".format(q3_bw))


print("Completed")

When performing whole-program analysis using DTrace, be sure to suppress the benchmark's output (bare `-B`) mode as follows (this prevents DTrace from tracing the printing of the benchmark):

In [ ]:
# Example benchmark
print("Running benchmark")

!io/io-static -B -q -r iofile

print("Completed")

The following example measures the performance of the read benchmark when the buffer cache is enabled and disable. As noted in the lab handouts, be sure to discard the ouput of the first run of the benchmark when the buffer cache is disabled (as cached data may still be  accessed):

In [ ]:
# Example benchmark command
print("Running benchmark")

enabled = !io/io-static -r iofile
enabled_again = !io/io-static -r iofile
disabled = !io/io-static -d -r iofile

print("Buf. cache enabled: bandwidth = {} KiBytes/sec".
      format(float(enabled[1].split()[1])))
print("Buf. cache enabled (2nd run): bandwidth = {} KiBytes/sec".
      format(float(enabled_again[1].split()[1])))
print("Buf. cache disabled: bandwidth = {} KiBytes/sec".
      format(float(disabled[1].split()[1])))

print("Completed")

To get a high-level summary of execution time (including a breakdown of total wall-clock time, time in userspace and "system-time") use the UNIX time command as follows:

In [ ]:
# Example benchmark command
print("Running benchmark")

!/usr/bin/time -p io/io-static -r -B -d -q iofile

print("Completed")

---
## DTrace

Experimentation with DTrace is best performed directly from within the shell:

```
ssh root@192.168.141.100
root@l41-beaglebone:~ # dtrace -n 'BEGIN { print("hello world"); }'
```

### python-dtrace

DTrace scripts can be ran within the Jupyter notebook with the assistance of the `python-dtrace` module. Note:  this is somewhat more involved that running the `dtrace` command from the shell as the script must be executed within a seperate thread.

The following example uses DTrace to measure the execution time of the benchmark:

In [ ]:
# D Language script
from dtrace import DTraceConsumerThread
io_performance_script = """
BEGIN {
   self->targetPid = -1;
}

proc:::exec-success
/execname == "io-static"/
{
   self->targetPid = pid;
   self->start = vtimestamp;
}

syscall::*exit:entry
/pid == self->targetPid/
{
   self->targetPid = -1;
   trace(vtimestamp - self->start);
}
"""

values = []

# Callback invoked to print the trace record
# (that is, printf("%u", vtimestamp - self->start))
def simple_out(value):
    values.append(int(value))
    
# Create a seperate thread to run the DTrace instrumentation
dtrace_thread = DTraceConsumerThread(io_performance_script,
                                    chew_func=lambda v: None,
                                    chewrec_func=lambda v: None,
                                    out_func=simple_out,
                                    walk_func=None,
                                    sleep=1)

# Start the DTrace instrumentation
dtrace_thread.start()

# Display header to indicate that the benchmarking has started
print("Starting io-static read performance measurement")

# Run the io-static benchmark    
TOTAL_SIZE = 16*1024*1024
BUFFER_SIZE = 8192

for trials in range(0, 10):
    output = !io/io-static -r -B -q -b {str(BUFFER_SIZE)} -t {str(TOTAL_SIZE)} iofile
        
# The benchmark has completed - stop the DTrace instrumentation
dtrace_thread.stop()
dtrace_thread.join()

# Compute and print the average time
average_time = sum(values[1:])/len(values[1:])
print("Average time = {} nsec".format(average_time))
print("Average bandwidth = {} KiBytes/sec".format((TOTAL_SIZE/1024)/(average_time/1e9)))

# Display footer to indicate that the benchmarking has finished
print("Finished io-static read performance measurement")

The following example uses DTrace to record the number of times each `syscall` is called by the `io-static` benchmark reading the benchmark data file:

In [ ]:
# D Language script
io_syscall_script = """
syscall:::entry
/execname == "io-static"/
{
    @a[probefunc] = count();
}
"""

# Callback invoked to process the aggregation
from collections import defaultdict
values = defaultdict(int)

def simple_out(a, b, c, d):
    values[c[0]] += d

# Create a seperate thread to run the DTrace instrumentation
dtrace_thread = DTraceConsumerThread(io_syscall_script,
                                    walk_func=simple_out,
                                    sleep=1)

# Start the DTrace instrumentation
dtrace_thread.start()

# Display header to indicate that the benchmarking has started
print("Starting io-static read performance measurement")

# Run the io-static benchmark    
TOTAL_SIZE = 16*1024*1024
BUFFER_SIZE = 512

output_dtrace = !io/io-static -r -b {str(BUFFER_SIZE)} -t {str(TOTAL_SIZE)} iofile
        
# The benchmark has completed - stop the DTrace instrumentation
dtrace_thread.stop()
dtrace_thread.join()
    
# Print the syscalls and their frequency
print("Number of read() calls {}".format(values['read']))

# Display footer to indicate that the benchmarking has finished
print("Finished io-static read performance measurement")

The following example explores the performance impact of adding a predicate clause in a DTrace script:

In [ ]:
# D Language scripts
io_syscall_script = """
syscall:::entry
/execname == "io-static"/
{
    @a[probefunc] = count();
}
"""

io_syscall_script_no_predicate = """
syscall:::entry
{
    @a[probefunc] = count();
}
"""

TOTAL_SIZE = 16*1024*1024
BUFFER_SIZE = 512
values = dict()

# Callback invoked to process the aggregation
def simple_out(a, b, c, d):
    key = c[0]
    count = d
    if key in values:
        values[key] += d
    else:
        values[key] = d

# Create a seperate thread to run the DTrace instrumentation
dtrace_thread = DTraceConsumerThread(io_syscall_script,
                                    chew_func=None,
                                    chewrec_func=None,
                                    walk_func=simple_out,
                                    sleep=1)

# Start the DTrace instrumentation
dtrace_thread.start()

# Display header to indicate that the benchmarking has started
print("Starting io-static read performance measurement")

# Run the io-static benchmark    
output_predicate = !io/io-static -r -b {str(BUFFER_SIZE)} -t {str(TOTAL_SIZE)} iofile
        
# The benchmark has completed - stop the DTrace instrumentation
dtrace_thread.stop()
dtrace_thread.join()

# Now run the benchmark again but without the predicate

# Create a seperate thread to run the DTrace instrumentation
dtrace_thread = DTraceConsumerThread(io_syscall_script_no_predicate,
                                    chew_func=None,
                                    chewrec_func=None,
                                    walk_func=simple_out,
                                    sleep=1)

# Start the DTrace instrumentation
dtrace_thread.start()

# Run the io-static benchmark    
output_no_predicate = !io/io-static -r -b {str(BUFFER_SIZE)} -t {str(TOTAL_SIZE)} iofile
        
# The benchmark has completed - stop the DTrace instrumentation
dtrace_thread.stop()
dtrace_thread.join()

# Print the performance of both runs
print("With predicate {} KiBytes/sec".format(output_predicate[1].split()[1]))
print("Without predicate {} KiBytes/sec".format(output_no_predicate[1].split()[1]))

# Display footer to indicate that the benchmarking has finished
print("Finished io-static read performance measurement")

---
## matplotlib and pandas

### Plot read performance (against buffer size)

The following example plots data read from a previous saved file. The example also demonstrates calcualtion of error bars using Python's `pandas` library. (Generating plots on the BeagleBone Black can be rather slow!)

In [ ]:
import json
import matplotlib
import pandas as pd
%matplotlib inline

# Plot the read performance (IO bandwidth against buffer size with error bars)
with open("2017-2018-l41-lab1.data", 'r') as f:
    content = f.readlines()
    
read_performance_values = [v.strip() for v in content]
       
# Buffer sizes to compute the performance with
BUFFER_SIZES = [2048 * 2 ** exp for exp in range(0, 14)]

# Total size of iofile (default size) = 16MiB
TOTAL_SIZE = BUFFER_SIZES[-1] #16*1024*1024

# Number of trials for each buffer size
NUM_TRIALS = 11

# Compute the IO bandwidth in KiBytes/sec
io_bandwidth_values = [(TOTAL_SIZE/1024)/(json.loads(val)["timestamp"]/1e9) for val in read_performance_values]

# Reshape the list into an array of size [len(BUFFER_SIZES), NUM_TRIALS]
io_bandwidth = np.reshape(io_bandwidth_values, (len(BUFFER_SIZES), NUM_TRIALS))[:,:]

# Convert the array of io bandwidth values into a Panda DataFrame
# this allows ploting of the median value and computation of the 
# error bars (25 and 75 percentile values)
# Note: The error bars should be small indicating that the experiment is tightly controlled
df = pd.DataFrame(io_bandwidth, index=BUFFER_SIZES)

# Compute error bars based on the 25 and 75 quartile values
error_bars = df.quantile([.25, .75], axis=1)
error_bars.loc[[0.25]] = df.median(1) - error_bars.loc[[0.25]]
error_bars.loc[[0.75]] = error_bars.loc[[0.75]] - df.median(1)
error_bars_values = [error_bars.values]

# Create and label the plot
plt.figure();
df.median(1).plot(figsize=(9,9), yerr=error_bars_values, label="io-static read")
plt.title('io-static read performance')
plt.ylabel('I/O bandwidth (KiBytes/sec)')
plt.xlabel('Buffer size (Bytes)')
plt.xscale('log')

# Plot a vertical line at 1MiB
plt.axvline(x=1024*1024, color='g')

# Display the plot and save it to a file
# (this can take a while (~30 secs) on the BeagleBone Black)
plt.savefig("2017-2018-l41-lab1-performance.pdf")

---